# Symmetric NPA relaxations

We initialize the RepLAB library.

In [1]:
addpath([pwd, '/../../..']);
replab_init('verbose', 0);

We examine an upper bound on the quantum maximum of the CHSH inequality. We use the NPA hierarchy, and assume the moment matrix has been constructed in the
symmetric subspace, as in [arXiv:1808.09598](https://arxiv.org/abs/1808.09598).

We first declare the one and only variable, before constructing the moment matrix.

In [2]:
y = sdpvar; % y is actually y_A0B0, but we save space by identifying y with it
C = [ 1  0  0  0  0
      0  1  0  0  0
      0  0  1  0  0
      0  0  0  1  0
      0  0  0  0  1];
A = [ 0  0  0  0  0
      0  0  0  1  1
      0  0  0  1 -1
      0  1  1  0  0
      0  1 -1  0  0];
X = C + A*y;
I_CHSH = 4*y; % it is <A0B0> + <A0B1> + <A1B0> - <A1B1>, and y_A1B1 = -y_A0B0.
optimize(X >= 0, -I_CHSH, sdpsettings('verbose', 0)); % sign change to maximize, and we don't show solver output
double(I_CHSH)

ans = 2.8284


To help with symmetrization, we'll use an equivalent form of the constraint X, and check the result.

Now, we want to find a change of basis that brings X into a block-diagonal form. This is easier to check on the basis matrices `C` and `A`, i.e. we want to find `U` such that `U*C*U'` and `U*A*U'` are block-diagonal. Let us use RepLAB for that.
First, remark that the moment matrix is invariant under the following signed permutations:

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, B_0, B_1, A_0, A_1 \right)$

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, -A_0, -A_1, -B_0, -B_1 \right)$

$\vec{v} = \left( 1, A_0, A_1, B_0, B_1 \right) \rightarrow  \left( 1, A_1, A_0, B_0, -B_1 \right)$

We now show how to use RepLAB to find the change of basis for the symmetry group that includes only the first symmetry.

In [3]:
g1 = [1 4 5 2 3]; % permutation of parties
g2 = [1 -2 -3 -4 -5]; % sign flip everywhere, note the signed permutation convention
g3 = [1 3 2 4 -5]; % additional symmetry

We build the symmetry group from those generators. For the permutation of parties only, the order (=size) of the group is 2. When adding the other symmetries, the order of the group should be 16.

In [4]:
nElements = 5;G = replab.SignedPermutationGroup.of(g1, g2, g3);
G.order

ans =
    16


If necessary, we can also expand the group elements (use `G.elements.at(2)` to get the second element for example).

In [5]:
G.elements

ans =


Indexed family of 16 elements
 1 = [1, 2, 3, 4, 5]
 2 = [1, -3, -2, -4, 5]
 3 = [1, 2, -3, 5, 4]
 4 = [1, 3, -2, -5, 4]
 5 = [1, 4, 5, 2, 3]
 6 = [1, -5, -4, -2, 3]
 7 = [1, 4, -5, 3, 2]
 8 = [1, 5, -4, -3, 2]
 9 = [1, -5, 4, 3, -2]
10 = [1, -4, 5, -3, -2]
11 = [1, 5, 4, 2, -3]
12 = [1, -4, -5, -2, -3]
13 = [1, -3, 2, 5, -4]
14 = [1, -2, 3, -5, -4]
15 = [1, 3, 2, 4, -5]
16 = [1, -2, -3, -4, -5]



Now, the representation we need is composed of signed permutation matrices; as we are considering a group of signed permutations, this is the group *natural representation*. Representations in RepLAB are described using the images of the generators. By calling `G.sample`, we get random elements from the group, and by calling `rep.image(g)` we get the image of a group element.

In [6]:
rep = G.naturalRep

rep =


Orthogonal representation
          dimension: 5
divisionAlgebraName: []
              field: 'R'
              group: replab.SignedPermutationGroup
   imagesErrorBound: [0, 0, 0]
          isUnitary: true
           morphism: replab.mrp.FiniteComposition
       preimages{1}: [1, 4, 5, 2, 3]
          images{1}: 5 x 5 double
       preimages{2}: [1, -2, -3, -4, -5]
          images{2}: 5 x 5 double
       preimages{3}: [1, 3, 2, 4, -5]
          images{3}: 5 x 5 double



In [7]:
g = G.sample

g =

   1   4  -5   3   2



Now, we decompose the representation into irreducible components. For that, we call `rep.decomposition`. In the answer I(m)xR(d), we express that the component has $m$ copies (multiplicity) of a `R`eal representation of dimension `d`. We then play with the indices of the component, and copy.

Hint: explore `rep.decomposition.component(i).irrep(j)` for $(i,j) = (1,1), (2,1), (3,1)$.

In [8]:
rep.decomposition

ans =

Orthogonal reducible representation
          dimension: 5
divisionAlgebraName: []
              field: 'R'
              group: replab.SignedPermutationGroup
 injection_internal: 5 x 5 double
       isSimilarRep: true
          isUnitary: true
     mapsAreAdjoint: true
             parent: Orthogonal representation
projection_internal: 5 x 5 double
  basis(1,'double'): [1; 0; 0; 0; 0]
  basis(2,'double'): [0; 0.69692; -0.11957; 0.40825; 0.57735]
  basis(3,'double'): [0; -0.11957; -0.69692; -0.57735; 0.40825]
  basis(4,'double'): [0; -0.32506; 0.62796; -0.21419; 0.67389]
  basis(5,'double'): [0; -0.62796; -0.32506; 0.67389; 0.21419]
       component(1): Isotypic component R(1) (trivial)
       component(2): Isotypic component R(2) (nontrivial)
       component(3): Isotypic component R(2) (nontrivial)



In [9]:
rep.decomposition.component(1).irrep(1)

ans =

Orthogonal trivial irreducible real-type subrepresentation
          dimension: 1
divisionAlgebraName: []
              field: 'R'
              group: replab.SignedPermutationGroup
 injection_internal: [1; 0; 0; 0; 0]
       isSimilarRep: false
          isUnitary: true
     mapsAreAdjoint: true
             parent: Orthogonal representation
projection_internal: [1, 0, 0, 0, 0]
   basis(1,'exact'): [1; 0; 0; 0; 0]



We now ask for the change of basis matrix, and verify that `A` and `C` block-diagonalize.

In [10]:
U = rep.decomposition.basis
U'*C*U
U'*A*U

U =

   1.0000        0        0        0        0
        0   0.6969  -0.1196  -0.3251  -0.6280
        0  -0.1196  -0.6969   0.6280  -0.3251
        0   0.4082  -0.5774  -0.2142   0.6739
        0   0.5774   0.4082   0.6739   0.2142



ans =

   1.0000        0        0        0        0
        0   1.0000   0.0000   0.0000   0.0000
        0   0.0000   1.0000   0.0000   0.0000
        0   0.0000   0.0000   1.0000  -0.0000
        0   0.0000   0.0000  -0.0000   1.0000



ans =

        0        0        0        0        0
        0   1.4142   0.0000  -0.0000   0.0000
        0   0.0000   1.4142  -0.0000  -0.0000
        0  -0.0000  -0.0000  -1.4142   0.0000
        0   0.0000  -0.0000   0.0000  -1.4142



Once we have found the change of basis matrix, we transform our moment matrix `X`, and naturally it should have a block diagonal structure following the one of `C` and `A` -- for that, it may be necessary to kill small off-block coefficients.

In [11]:
X = (U'*C*U) + (U'*A*U)*y; % equivalent to the constraint above
optimize(X >= 0, -I_CHSH); % sign change to maximize
double(I_CHSH)


 num. of constraints =  1


 dim. of sdp    var  =  5,   num. of sdp  blk  =  1  1th semidefinite block is actually diagonal



*******************************************************************
   SDPT3: Infeasible path-following algorithms
*******************************************************************
 version  predcorr  gam  expon  scale_data
   HKM      1      0.000   1        0    
it pstep dstep pinfeas dinfeas  gap      prim-obj      dual-obj    cputime
-------------------------------------------------------------------
 0|0.000|0.000|8.0e-01|6.2e+00|5.0e+02| 5.000000e+01  0.000000e+00| 0:0:00| chol  1  1 
 1|1.000|1.000|6.5e-08|6.9e-02|4.3e+01| 3.984640e+01  4.400004e-01| 0:0:00| chol  1  1 
 2|0.933|1.000|1.5e-07|6.9e-03|2.9e+00| 3.700414e+00  8.665599e-01| 0:0:00| chol  1  1 
 3|1.000|0.949|3.5e-08|1.0e-03|2.8e-01| 3.024905e+00  2.752468e+00| 0:0:00| chol  1  1 
 4|0.987|0.988|1.1e-08|8.0e-05|3.4e-03| 2.830875e+00  2.827762e+00| 0:0:00| chol  1  1 
 5|0.989|0.989|3.5e-10|7.7e-06|3.8e-05| 2.828454e+00  2.828448e+00| 0:0:00| chol  1  1 
 6|0.989|0.989|8.9e-12|8.5e-08|4.2e-07| 2.828427e+00  2.828

ans = 2.8284


As an exercice, recover an algebraic basis from `U`, guess the form of the semidefinite program when fully block-diagonal (i.e. for the full group of CHSH symmetries).
**Remark that now the problem can be solved by hand to recover the** $2\sqrt{2}$ **bound, as the SDP matrix is fully diagonal, and thus corresponds to linear inequalities.**